In [2]:
#imports 

import numpy as np
import os, tempfile
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import Sequential

from sklearn.dummy import DummyClassifier

import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from tensorflow.keras.preprocessing import image_dataset_from_directory

import joblib

## Image Load In

In [3]:
!ls

Data
Peter.ipynb
PeterNotebook.ipynb
README.md
VGG19.ipynb
Yuhkai_notebook_scratch.ipynb


In [4]:
!cd ..

In [5]:
!ls

Data
Peter.ipynb
PeterNotebook.ipynb
README.md
VGG19.ipynb
Yuhkai_notebook_scratch.ipynb


In [6]:
!ls

Data
Peter.ipynb
PeterNotebook.ipynb
README.md
VGG19.ipynb
Yuhkai_notebook_scratch.ipynb


In [7]:
# load in the train data

#starting from main project directory

train_normal_dir = "./data/chest_xray/train/NORMAL"
train_pneumonia_dir = "./data/chest_xray/train/PNEUMONIA"

imgs_train_normal = [file for file in os.listdir(train_normal_dir) if file.endswith('.jpeg')]
imgs_train_pneumonia = [file for file in os.listdir(train_pneumonia_dir) if file.endswith('.jpeg')]

In [8]:
test_normal_dir = "./data/chest_xray/test/NORMAL"
test_pneumonia_dir = "./data/chest_xray/test/PNEUMONIA"

imgs_test_normal = [file for file in os.listdir(test_normal_dir) if file.endswith('.jpeg')]
imgs_test_pneumonia = [file for file in os.listdir(test_pneumonia_dir) if file.endswith('.jpeg')]

In [9]:
len(imgs_train_normal)

1341

In [10]:
len(imgs_train_pneumonia)

3875

In [11]:
imgs_train_normal[0]

'IM-0115-0001.jpeg'

In [12]:
imgs_train_pneumonia[0]

'person1000_bacteria_2931.jpeg'

In [13]:
len(imgs_test_normal)

234

In [14]:
len(imgs_test_pneumonia)

390

In [15]:
num_train = len(imgs_train_normal) + len(imgs_train_pneumonia)
num_test = len(imgs_test_normal) + len(imgs_test_pneumonia)
num_train, num_test

(5216, 624)

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [17]:
#rescale images to 64 by 64
#create validation set as 20% of train set

train_folder = "./data/chest_xray/train"
train_gen = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
       
        class_mode='binary',
        subset='training',
        batch_size= num_train)

validation_gen = train_datagen.flow_from_directory(
        train_folder, 
        target_size=(64, 64), 
       
        class_mode='binary',
        subset='validation',
        batch_size= num_train)

test_folder = "./data/chest_xray/test"
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_folder, 
        target_size=(64, 64), 
        class_mode='binary', 
        batch_size= num_test)

Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [18]:
train_images, train_labels = next(train_gen)
val_images, val_labels = next(validation_gen)
test_images, test_labels = next(test_gen)

In [19]:
print(train_images.shape)

(4173, 64, 64, 3)


In [20]:
train_images[0]

array([[[0.04313726, 0.04313726, 0.04313726],
        [0.15686275, 0.15686275, 0.15686275],
        [1.        , 1.        , 1.        ],
        ...,
        [0.12941177, 0.12941177, 0.12941177],
        [0.1137255 , 0.1137255 , 0.1137255 ],
        [0.08627451, 0.08627451, 0.08627451]],

       [[0.07058824, 0.07058824, 0.07058824],
        [0.10196079, 0.10196079, 0.10196079],
        [0.12941177, 0.12941177, 0.12941177],
        ...,
        [0.12941177, 0.12941177, 0.12941177],
        [0.10980393, 0.10980393, 0.10980393],
        [0.09803922, 0.09803922, 0.09803922]],

       [[0.10588236, 0.10588236, 0.10588236],
        [0.12156864, 0.12156864, 0.12156864],
        [0.13725491, 0.13725491, 0.13725491],
        ...,
        [0.13333334, 0.13333334, 0.13333334],
        [0.1137255 , 0.1137255 , 0.1137255 ],
        [0.08627451, 0.08627451, 0.08627451]],

       ...,

       [[0.07058824, 0.07058824, 0.07058824],
        [0.07450981, 0.07450981, 0.07450981],
        [0.07058824, 0

In [21]:
val_images[0]

array([[[0.07843138, 0.07843138, 0.07843138],
        [0.07843138, 0.07843138, 0.07843138],
        [0.06666667, 0.06666667, 0.06666667],
        ...,
        [0.72156864, 0.72156864, 0.72156864],
        [0.7254902 , 0.7254902 , 0.7254902 ],
        [0.7725491 , 0.7725491 , 0.7725491 ]],

       [[0.03921569, 0.03921569, 0.03921569],
        [0.03921569, 0.03921569, 0.03921569],
        [0.03529412, 0.03529412, 0.03529412],
        ...,
        [0.72156864, 0.72156864, 0.72156864],
        [0.7294118 , 0.7294118 , 0.7294118 ],
        [0.7019608 , 0.7019608 , 0.7019608 ]],

       [[0.04705883, 0.04705883, 0.04705883],
        [0.04313726, 0.04313726, 0.04313726],
        [0.13333334, 0.13333334, 0.13333334],
        ...,
        [0.7372549 , 0.7372549 , 0.7372549 ],
        [0.7137255 , 0.7137255 , 0.7137255 ],
        [0.7137255 , 0.7137255 , 0.7137255 ]],

       ...,

       [[0.04313726, 0.04313726, 0.04313726],
        [0.04313726, 0.04313726, 0.04313726],
        [0.04313726, 0

In [22]:
#(image height, image width, color channels)
train_images[0].shape

(64, 64, 3)

In [23]:
#preserving shape for VGG19


## VGG19

In [49]:
from keras.applications import VGG19
from keras import models
from keras import layers
from keras import optimizers


cnn_base = VGG19(weights='imagenet', 
                 include_top=False, 
                 input_shape=(64, 64, 3))

In [50]:
VGG_model = models.Sequential()
VGG_model.add(cnn_base)

In [51]:
VGG_model.compile(loss='binary_crossentropy',
                optimizer='RMSprop',
                metrics=['accuracy', tf.keras.metrics.Precision(),  tf.keras.metrics.Recall()])

In [58]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping 

stop = EarlyStopping(monitor='loss', mode='min', patience=5)
callbacks = [stop]

In [59]:
VGG_model.fit(train_images, train_labels, epochs=20, batch_size=50, callbacks = [stop],validation_data=(val_images, val_labels))

Epoch 1/20


TypeError: 'NoneType' object is not callable

In [ ]:
## accuracy: 0.9827 - precision_9: 0.9878 - recall_1: 0.9890

## VGG19 with added layers

In [54]:
VGGtransfer_model = models.Sequential()
VGGtransfer_model.add(cnn_base)
VGGtransfer_model.add(layers.Flatten())
VGGtransfer_model.add(layers.Dense(128, activation='relu'))
VGGtransfer_model.add(layers.Dense(1, activation='sigmoid'))

In [55]:
cnn_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [60]:
VGGtransfer_model.compile(loss='binary_crossentropy',
                optimizer='RMSprop',
                metrics=['accuracy', tf.keras.metrics.Precision(),  tf.keras.metrics.Recall()])

In [61]:
VGGtransfer_model.fit(train_images, train_labels, epochs=20, batch_size=100, callbacks = [stop], validation_data=(val_images, val_labels))

Epoch 1/20
 1/42 [..............................] - ETA: 0s - loss: 0.8207 - accuracy: 0.7500 - precision_6: 0.7500 - recall_6: 1.0000

KeyboardInterrupt: 

In [ ]:
## transfer_model.fit(train_images, train_labels, epochs=10, batch_size=30, callbacks = [stop])=accuracy: 0.9286 - precision_7: 0.9572